# Text Generation Apps

Link to TDS [article](https://towardsdatascience.com/build-a-text-generator-web-app-in-under-50-lines-of-python-9b63d47edabb)

#### This notebook contains two variations of the text generation application app. 
- Basic Application
- Advanced Application with added probabilities 

## Part 1: Setting up the Model

In [2]:
pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.2 MB/s eta 0:00:00


In [3]:
# Loading model dependencies
import numpy as np
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from random import choice

In [4]:
# Downloading the model
tok = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

### Writing the prediction function

In [5]:
def get_pred(text, model, tok, p=0.7):
    # 1. tokenize/encode the input text
    input_ids = torch.tensor(tok.encode(text)).unsqueeze(0)
    # 2. extract the logits vector for the next possible token
    logits = model(input_ids)[0][:, -1]
    # 3. apply softmax to the logits so we have the probabilities of each word add up to 1
    probs = F.softmax(logits, dim=-1).squeeze()
    # 4. sort the probabilities in descending order 
    idxs = torch.argsort(probs, descending=True)
    # 5. loop through the ordered probabilities until they sum up to p. Then, randomly choose an option
    res, cumsum = [], 0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum > p:
            pred_idx = idxs.new_tensor([choice(res)])
            break
    # 6. convert the chosen prediction into text
    pred = tok.convert_ids_to_tokens(int(pred_idx))
    return tok.convert_tokens_to_string(pred)

In [6]:
# testing it out
get_pred("wow this tutorial is", model, tok, p = 0.7) 

' quite'

## Basic Application

In [7]:
import panel as pn
pn.extension() # loading panel's extension for jupyter compatibility

In [8]:
# creating the text input widget
text_input = pn.widgets.TextInput(placeholder="Input starting text here"); text_input

TextInput(placeholder='Input starting t...)

In [9]:
# creating the markdown text pane where generated text will go
generated_text = pn.pane.Markdown(object=text_input.value)

In [10]:
# linking text_input to generated_text so the latter updates everytime the former changes
text_input.link(generated_text, value='object')

Watcher(inst=TextInput(placeholder='Input starting t...), cls=<class 'panel.widgets.input.TextInput'>, fn=<function Reactive.link.<locals>.link_cb at 0x7f7bc46410d0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [11]:
# previewing the contents of both and observing how one changes the other (have to press enter after inputting text)
pn.Row(text_input, generated_text)

Row
    [0] TextInput(placeholder='Input starting t...)
    [1] Markdown(str)

In [12]:
# create the button widget
button = pn.widgets.Button(name="Generate",button_type="primary")

In [13]:
# writing the call back function when the generate_button is clicked
def click_cb(event):
    pred = get_pred(generated_text.object, model, tok)
    generated_text.object += pred

In [14]:
# linking the on_click acton with the click_cb function
button.on_click(click_cb)

Watcher(inst=Button(button_type='primary', name='Generate'), cls=<class 'panel.widgets.button.Button'>, fn=<function click_cb at 0x7f7bc45505e0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [15]:
# compiling our app with the objects we have created thus far
app = pn.Column(pn.Column(text_input,button,generated_text)); app

Traceback (most recent call last):
 File "/usr/local/lib/python3.9/dist-packages/pyviz_comms/__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.9/dist-packages/panel/viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.9/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/util/callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.9/dist-packages/bokeh/util/callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.9/dist-packages/panel/io/state.py", line 408, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.9/dist-packages/panel/widgets/button.py", line 184, in _process_event
 self.clicks += 1
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 367, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/__init__.py", line 625, in __set__
 super(Dynamic,self).__set__(obj,val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 1252, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 2043, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-13-d7cee4d6f568>", line 3, in click_cb
 pred = get_pred(generated_text.object, model, tok)
 File "<ipython-input-5-acba27e0f868>", line 5, in get_pred
 logits = model(input_ids)[0][:, -1]
 File "/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py", line 1194, in _call_impl
 return forward_call(*input, **kwargs)
 File "/usr/local/lib/python3.9/dist-packages/transformers/models/gpt2/modeling_gpt2.py", line 1043, in forward
 transformer_outputs = self.transformer(
 File "/usr/local/lib/python3.9/dist-packag

Column
    [0] Column
        [0] TextInput(placeholder='Input starting t...)
        [1] Button(button_type='primary', name='Generate')
        [2] Markdown(str)

In [16]:
# Writing a title and description
title = pn.pane.Markdown("# **Text Generator**")
desc = pn.pane.HTML("<marquee scrollamount='10'><b>Welcome to the text generator! In order to get started, simply enter some starting input text below, click generate a few times and watch it go!</b></marquee>")

# Defining the final application and running it
final_app = pn.Column(title, desc ,app); final_app

Traceback (most recent call last):
 File "/usr/local/lib/python3.9/dist-packages/pyviz_comms/__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.9/dist-packages/panel/viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.9/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/util/callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.9/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.9/dist-packages/bokeh/util/callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.9/dist-packages/panel/io/state.py", line 408, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.9/dist-packages/panel/reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.9/dist-packages/panel/widgets/button.py", line 184, in _process_event
 self.clicks += 1
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 367, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/__init__.py", line 625, in __set__
 super(Dynamic,self).__set__(obj,val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 1252, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 2043, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.9/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "<ipython-input-13-d7cee4d6f568>", line 3, in click_cb
 pred = get_pred(generated_text.object, model, tok)
 File "<ipython-input-5-acba27e0f868>", line 5, in get_pred
 logits = model(input_ids)[0][:, -1]
 File "/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py", line 1194, in _call_impl
 return forward_call(*input, **kwargs)
 File "/usr/local/lib/python3.9/dist-packages/transformers/models/gpt2/modeling_gpt2.py", line 1043, in forward
 transformer_outputs = self.transformer(
 File "/usr/local/lib/python3.9/dist-packag

Column
    [0] Markdown(str)
    [1] HTML(str)
    [2] Column
        [0] Column
            [0] TextInput(placeholder='Input starting t...)
            [1] Button(button_type='primary', name='Generate')
            [2] Markdown(str)

In [17]:
# this command will launch the final app in a new tab (for debugging purposes)
# final_app.show()

In [18]:
# this command is needed in order to serve this app in production mode. (make sure to uncomment ofcourse)
# final_app.servable()

In [19]:
# bash command to serve this app in a production setting
# ! panel serve --show text_generation_app.ipynb

## Advanced Application

Now, we will built upon the basic application and add some more features, namely the ability for the user to select a token out a list of most probably vectors. This is standalone from the code of the previous application. We are utilizing the model and tokenizer ofcourse. 

In [20]:
# Redefining the predictions function since we now want to return a list of most likely next tokens 
# instead of a single token. Also, we want to return the proabilities in order to return them to 
# the user as well.

def get_preds(text, model, tok, p=0.7):
    input_ids = torch.tensor(tok.encode(text)).unsqueeze(0)
    logits = model(input_ids)[0][:, -1]
    probs = F.softmax(logits, dim=-1).squeeze()
    idxs = torch.argsort(probs, descending=True)
    res,pred_probs = [],[]
    for idx in idxs:
        res.append(idx)
        pred_probs.append(probs[idx])
        if sum(pred_probs) > p:
            pred_idxs = [idxs.new_tensor([p]) for p in res]
            break
    preds = [tok.convert_ids_to_tokens(int(p)) for p in pred_idxs]
    return [tok.convert_tokens_to_string(pred) for pred in preds], pred_probs

In [21]:
text_input = pn.widgets.TextInput(value="",width=400)
generated_text = pn.pane.Markdown(text_input.value)
start_button = pn.widgets.Button(name="Generate",button_type="primary")

# creating radio buttons for the token options along with probabilities 
options = [""]
radio_button = pn.widgets.RadioButtonGroup(options=options,height=30,width=500)
prob_button = pn.widgets.RadioButtonGroup(options=options,height=30,width = 500)

# since the prob_button is only to inform the user of the probabilities, we don't need to be enabled
prob_button.disabled=True

# new click callback function which handles the updation of the radio buttons
def click_cb(event):
    if radio_button.value == "<|endoftext|>": 
        start_button.disabled = True
        return None
    generated_text.object += radio_button.value
    preds, probs = get_preds(generated_text.object, model, tok)
    radio_button.options = preds[:10]
    radio_button.value = radio_button.options[np.random.randint(0,len(radio_button.options))]
    prob_button.options = [str(round(float(i),2)) for i in probs[:10]]

start_button.on_click(click_cb)

# call back function in case the text input changes. Essentially, we need to reset our options. 
def text_change_cb(event):
    generated_text.object = event.new
    start_button.disabled = False
    radio_button.options = options
    radio_button.value = radio_button.options[0]
    prob_button.options = options

# tying the callback function to the text_input widget
text_input.param.watch(text_change_cb, 'value')

# preparing the app
app = pn.Column(text_input,radio_button,prob_button,start_button,generated_text)

In [22]:
# Panel spacer object to center our title
h_spacer = pn.layout.HSpacer()

# defining the title and description 
title = pn.pane.Markdown("# **Text Generator**")
desc = pn.pane.HTML("<i>Welcome to the text generator! In order to get started, simply enter some starting input text below, click generate a few times and watch it go! You can also choose to select which token gets chosen using the radio buttons. Probabilities for each of which can be seen underneath. Give it a shot!</i>")

In [23]:
# setting up the final app
final_app = pn.Column(pn.Row(h_spacer,title,h_spacer), desc ,app); final_app

Column
    [0] Row
        [0] HSpacer()
        [1] Markdown(str)
        [2] HSpacer()
    [1] HTML(str)
    [2] Column
        [0] TextInput(width=400)
        [1] RadioButtonGroup(height=30, options=[''], sizing_mode='fixed', width=500)
        [2] RadioButtonGroup(disabled=True, height=30, options=[''], sizing_mode='fixed', width=500)
        [3] Button(button_type='primary', name='Generate')
        [4] Markdown(str)

In [24]:
# this command will launch the final app in a new tab (for debugging purposes)
# final_app.show()

In [25]:
# this command is needed in order to serve this app in production mode. (make sure to uncomment ofcourse)
final_app.servable()

Column
    [0] Row
        [0] HSpacer()
        [1] Markdown(str)
        [2] HSpacer()
    [1] HTML(str)
    [2] Column
        [0] TextInput(width=400)
        [1] RadioButtonGroup(height=30, options=[''], sizing_mode='fixed', width=500)
        [2] RadioButtonGroup(disabled=True, height=30, options=[''], sizing_mode='fixed', width=500)
        [3] Button(button_type='primary', name='Generate')
        [4] Markdown(str)

In [26]:
# bash command to serve this app in a production setting
# ! panel serve --show text_generation_app.ipynb